In [1]:
from collections import Counter, deque

import numpy as np
from matplotlib import pyplot as plt
from scipy.sparse import coo_matrix
from sklearn.cluster import AgglomerativeClustering
from KernelTreelets import kernel_treelets_SVC

In [3]:
graph = np.genfromtxt(r"fb.txt", dtype=np.uint16)
adjmat_int8 = coo_matrix((np.ones(graph.shape[0], dtype=np.uint16), graph.T), shape=(4039, 4039), dtype=np.int8).toarray()
adjmat_int8 = adjmat_int8 + adjmat_int8.T
adjmat = np.array(adjmat_int8, dtype=float)
adjmat[np.diag_indices_from(adjmat)] = 1045

In [4]:
ac_ward = AgglomerativeClustering(compute_full_tree=True, linkage='ward').fit(adjmat)

In [5]:
ac_complete = AgglomerativeClustering(compute_full_tree=True, linkage='complete').fit(adjmat)

In [ ]:
ac_average = AgglomerativeClustering(compute_full_tree=True, linkage='average').fit(adjmat)

In [ ]:
ac_single = AgglomerativeClustering(compute_full_tree=True, linkage='single').fit(adjmat)

In [ ]:
class network_kernel:
	def __init__ (self, diag):
		self.diag = diag

	def __call__ (self, X):
		X = X[:, np.asarray(X).max(axis=0) > 1]
		print(X)
		X = X.copy()
		X[np.diag_indices_from(X)] = self.diag
		return X

ktc = kernel_treelets_SVC(network_kernel(1045), number_of_clusters=30, max_sample=4039, verbose=True)
ktc.fit(adjmat)

In [ ]:
def confmat(graph, tree):
    E = len(graph)
    V = np.max(graph) + 1
    db = np.zeros(V, dtype=int)
    a4 = np.zeros(V, dtype=int)
    adjmat_int = np.asarray(adjmat_int8, dtype=int)
    ufg = {x: np.zeros(V, dtype=bool) for x in range(V)}
    for x in range(V):
        ufg[x][x] = True
    for i in range(V - 1):
        p = tree[i][0]
        q = tree[i][1]
        db[i + 1] = 2 * ufg[p].sum() * ufg[q].sum()
        a4[i + 1] = 2 * adjmat_int[ufg[p]][:, ufg[q]].sum()
        ufg[i + V] = np.logical_or(ufg[p], ufg[q])
        ufg.pop(p)
        ufg.pop(q)

    a4 = np.cumsum(a4)
    db = np.cumsum(db)
    a2 = db - a4
    # a1 = V * V - 2 * E - a2 - V 
    # a3 = 2 * E - a4
    tp = a4 / 2 / E
    fp = a2 / (V * V - 2 * E - V)
    return (fp, tp)

In [ ]:
L = confmat(graph, ac_single.children_)
acs = [ktc, ac_ward, ac_complete, ac_average, ac_single]

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("FB Network Data ROC curve (core=1045)")
pairs = [confmat(graph, x.children_) for x in acs]
methods = [plt.plot(x[0], x[1]) for x in pairs]
names = ["KT", "Ward", "Complete", "Average", "Single"]
auc = [1 - np.trapz(x[0], x[1]) for x in pairs]
plt.legend([i[0] for i in methods], [names[i] + ": AUC = {:.3f}".format(auc[i]) for i in range(len(methods))])

plt.show()